In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OrdinalEncoder

In [2]:
X = datasets.fetch_covtype().data[:5000]
y = datasets.fetch_covtype().target[:5000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(X_train.shape, y_train.shape)

enc = OrdinalEncoder()
y_train = enc.fit_transform(y_train.reshape(-1, 1)).reshape(-1, )
y_test = enc.transform(y_test.reshape(-1, 1)).reshape(-1, )
print(np.unique(y_train))  # 7分类任务

X_train, X_test, y_train, y_test = tf.constant(X_train), tf.constant(X_test), tf.constant(y_train), tf.constant(y_test)

(4500, 54) (4500,)
[0. 1. 2. 3. 4. 5. 6.]


In [3]:
inputs = tf.keras.Input(shape=(54,))
x = tf.keras.layers.Dense(256, activation='tanh')(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128)(x)
predictions = tf.keras.layers.Dense(7)(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 54)]              0         
                                                                 
 dense (Dense)               (None, 256)               14080     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 7)                 903       
                                                                 
Total params: 64,391
Trainable params: 64,391
Non-trainable p

In [4]:
# Configures the model for training.
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # 优化器
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # 损失函数
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])  # 评估函数(根据模型的输入与输出选择)

In [5]:
# Trains the model for a fixed number of epochs (iterations on a dataset).
'''
x:  Input data. It could be:
    A Numpy array (or array-like), or a list of arrays (in case the model has multiple inputs).
    A TensorFlow tensor, or a list of tensors (in case the model has multiple inputs).
    A dict mapping input names to the corresponding array/tensors, if the model has named inputs.
    A tf.data dataset. Should return a tuple of either (inputs, targets) or (inputs, targets, sample_weights).
    A generator or keras.utils.Sequence returning (inputs, targets) or (inputs, targets, sample_weights).
y:  Target data. Like the input data x, it could be either Numpy array(s) or TensorFlow tensor(s).
    It should be consistent with x (you cannot have Numpy inputs and tensor targets, or inversely).
    If x is a dataset, generator, or keras.utils.Sequence instance, y should not be specified (since targets will be obtained from x).
batch_size:  Integer or None.
             Number of samples per gradient update.
             If unspecified, batch_size will default to 32.
             Do not specify the batch_size if your data is in the form of datasets, generators, or keras.utils.Sequence instances (since they generate batches).
shuffle:  Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
          This argument is ignored when x is a generator or an object of tf.data.Dataset.
          'batch' is a special option for dealing with the limitations of HDF5 data;
          it shuffles in batch-sized chunks. Has no effect when steps_per_epoch is not None.

validation_split:  Float between 0 and 1. Fraction of the training data to be used as validation data.
                   The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch.
                   The validation data is selected from the last samples in the x and y data provided, before shuffling.
                   This argument is not supported when x is a dataset, generator or keras.utils.Sequence instance.
                   validation_split is not yet supported with tf.distribute.experimental.ParameterServerStrategy.

validation_data:  Data on which to evaluate the loss and any model metrics at the end of each epoch.
                  The model will not be trained on this data. Thus, note the fact that the validation loss of data provided using validation_split or validation_data is not affected by regularization layers like noise and dropout.
                  validation_data will override validation_split. validation_data could be:
                  * A tuple (x_val, y_val) of Numpy arrays or tensors.
                  * A tuple (x_val, y_val, val_sample_weights) of NumPy arrays.
                  * A tf.data.Dataset.
                  * A Python generator or keras.utils.Sequence returning (inputs, targets) or (inputs, targets, sample_weights). validation_data is not yet supported with tf.distribute.experimental.ParameterServerStrategy.
'''
model.fit(
    # ★★★★★x的输入为tf.data.Dataset格式时,参数y、shuffle、batch_size不需要再指定(tf.data.Dataset中已包含)
    x=X_train, y=y_train, shuffle=True, batch_size=64,
    epochs=10,  # Integer. Number of epochs to train the model.
    # 'auto', 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases
    verbose=2,
    validation_data=(X_test, y_test),
    # validation_split=0.2 # 验证数据集占训练数据集的比例,取值范围为0~1
)

Epoch 1/10
71/71 - 1s - loss: 1.5064 - sparse_categorical_accuracy: 0.4224 - val_loss: 1.1983 - val_sparse_categorical_accuracy: 0.5720 - 1s/epoch - 17ms/step
Epoch 2/10
71/71 - 0s - loss: 1.2740 - sparse_categorical_accuracy: 0.4811 - val_loss: 1.1930 - val_sparse_categorical_accuracy: 0.5620 - 180ms/epoch - 3ms/step
Epoch 3/10
71/71 - 0s - loss: 1.2033 - sparse_categorical_accuracy: 0.5053 - val_loss: 1.1079 - val_sparse_categorical_accuracy: 0.5800 - 193ms/epoch - 3ms/step
Epoch 4/10
71/71 - 0s - loss: 1.1852 - sparse_categorical_accuracy: 0.5151 - val_loss: 1.0900 - val_sparse_categorical_accuracy: 0.5660 - 197ms/epoch - 3ms/step
Epoch 5/10
71/71 - 0s - loss: 1.1576 - sparse_categorical_accuracy: 0.5282 - val_loss: 1.0824 - val_sparse_categorical_accuracy: 0.5920 - 192ms/epoch - 3ms/step
Epoch 6/10
71/71 - 0s - loss: 1.1332 - sparse_categorical_accuracy: 0.5429 - val_loss: 1.0285 - val_sparse_categorical_accuracy: 0.6120 - 184ms/epoch - 3ms/step
Epoch 7/10
71/71 - 0s - loss: 1.1300

In [6]:
model.history.history.items()

dict_items([('loss', [1.5064438581466675, 1.273985505104065, 1.2033445835113525, 1.1852405071258545, 1.1576391458511353, 1.133159875869751, 1.1300488710403442, 1.1259403228759766, 1.0705616474151611, 1.1086022853851318]), ('sparse_categorical_accuracy', [0.4224444329738617, 0.4811111092567444, 0.5053333044052124, 0.5151110887527466, 0.5282222032546997, 0.542888879776001, 0.5344444513320923, 0.5393333435058594, 0.5688889026641846, 0.54666668176651]), ('val_loss', [1.1983457803726196, 1.1929835081100464, 1.1079190969467163, 1.0899882316589355, 1.0823750495910645, 1.028539776802063, 1.084852695465088, 0.9655709266662598, 0.9374366998672485, 0.9748281836509705]), ('val_sparse_categorical_accuracy', [0.5720000267028809, 0.5619999766349792, 0.5799999833106995, 0.5659999847412109, 0.5920000076293945, 0.6119999885559082, 0.5860000252723694, 0.628000020980835, 0.6359999775886536, 0.6359999775886536])])

In [7]:
# Returns the loss value & metrics values for the model in test mode.
'''
batch_size:Integer or None. Number of samples per batch of computation. If unspecified, batch_size will default to 32. Do not specify the batch_size if your data is in the form of a dataset, generators, or keras.utils.Sequence instances (since they generate batches).
verbose:0 or 1. Verbosity mode. 0 = silent, 1 = progress bar.
'''
loss, metric = model.evaluate(X_test, y_test, batch_size=32, verbose=0)

print("test loss:", loss)
print("test acc:", metric)

test loss: 0.9748282432556152
test acc: 0.6359999775886536


In [8]:
# Generates output predictions for the input samples.
'''
batch_size:Integer or None. Number of samples per batch. If unspecified, batch_size will default to 32. Do not specify the batch_size if your data is in the form of dataset, generators, or keras.utils.Sequence instances (since they generate batches).
verbose:Verbosity mode, 0 or 1.
'''
predictions = model.predict(X_test[:10],  # 预测10条样本
                            batch_size=32, verbose=0)
print('predictions shape:', predictions)

predictions shape: [[-2.1469142e+00  2.8040659e-01  1.6221796e+00  2.2470996e+00
   7.5516933e-01  1.3969027e+00 -4.4346781e+00]
 [-3.5667901e+00 -5.5741332e-03  2.8905621e+00 -5.8800167e-01
   2.1176803e+00  2.2385547e+00 -5.7242589e+00]
 [-1.6141566e+00  2.5832698e-01  1.1618479e+00  3.4982401e-01
  -1.2971216e-01  1.5149839e+00 -3.4997342e+00]
 [-4.8402553e+00 -7.7331239e-01  4.2455177e+00  2.7187142e+00
   1.6327490e+00  2.7895849e+00 -7.4197755e+00]
 [ 8.6835194e-01  4.1537166e+00 -8.7388647e-01 -6.4929968e-01
   7.6327486e+00 -4.9395636e-01 -5.3424716e+00]
 [ 4.9506507e+00  6.3273096e+00 -3.9324994e+00 -3.1245191e+00
   6.7442697e-01 -4.1873379e+00  1.3151098e+00]
 [-2.5609262e+00 -2.9262003e-02  2.4307187e+00  2.0010431e+00
   6.2011433e-01  2.3030858e+00 -5.7678533e+00]
 [ 2.3241200e+00  2.1024721e+00 -2.6059020e+00 -2.8757370e+00
   1.4059093e+00 -2.3189299e+00  3.2203088e+00]
 [-1.9971784e+00  2.1660222e-01  9.5035160e-01  3.7708831e+00
   2.5086462e-01  1.2604591e+00 -4.3579